In [73]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



train = pd.read_csv("C:/Users/inves/Anaconda/Prediction Estate Price/home-data-for-ml-course/train.csv")
test = pd.read_csv("C:/Users/inves/Anaconda/Prediction Estate Price/home-data-for-ml-course/test.csv")


print(train.info())
#print(train.describe())

missing_values = train.isnull().sum().sort_values(ascending=False)
#print(missing_values[missing_values>0])
#print("------------------")
for element in missing_values:
    print(element)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [75]:
for col in train.select_dtypes(include=['int64', 'float64']).columns:
    train[col] = train[col].fillna(train[col].median())

for col in test.select_dtypes(include=['int64', 'float64']).columns:
    test[col] = test[col].fillna(test[col].median())

for col in train.select_dtypes(include=['object']).columns:
    train[col] = train[col].fillna("None")

for col in test.select_dtypes(include=['object']).columns:
    test[col] = test[col].fillna("None")

In [77]:
missing_values = train.isnull().sum().sort_values(ascending=False)
for element in missing_values:
    print(element)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [79]:
# Create new feature: House Age
train["HouseAge"] = train["YrSold"] - train["YearBuilt"]
test["HouseAge"] = test["YrSold"] - test["YearBuilt"]

# Create Total Bathrooms
train["TotalBathrooms"] = train["FullBath"] + (train["HalfBath"] * 0.5)
test["TotalBathrooms"] = test["FullBath"] + (test["HalfBath"] * 0.5)

# Log-transform the target variable (SalePrice)
train["SalePrice"] = np.log1p(train["SalePrice"])

In [81]:
from sklearn.preprocessing import LabelEncoder

# Label encode categorical features
categorical_cols = train.select_dtypes(include=["object"]).columns

for col in categorical_cols:
    le = LabelEncoder()

    # Fill missing values without inplace modification
    train[col] = train[col].fillna("Unknown")
    test[col] = test[col].fillna("Unknown")

    # Fit on train and transform both train and test
    train[col] = le.fit_transform(train[col])

    # Handle unseen labels in test by mapping with classes_, avoiding KeyError
    test[col] = test[col].apply(lambda s: le.transform([s])[0] if s in le.classes_ else -1)


In [82]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
#import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Define target and features
X = train.drop(["Id", "SalePrice"], axis=1)
y = train["SalePrice"]
X_test = test.drop(["Id"], axis=1)

# Split data for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train models
ridge = Ridge(alpha=1.0)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
xgb = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=4, random_state=42)

models = {"Ridge": ridge, "RandomForest": rf, "XGBoost": xgb}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    print(f"{name} RMSE: {rmse:.4f}")

Ridge RMSE: 0.1556
RandomForest RMSE: 0.1446
XGBoost RMSE: 0.1362


In [85]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Ensure X_test is a copy to avoid modifying the original dataframe
X_test = X_test.copy()

# Identify categorical columns
categorical_cols = X_test.select_dtypes(include=["object"]).columns

# Apply Label Encoding (same as used during training)
for col in categorical_cols:
    le = LabelEncoder()
    le.fit(train[col])  # Fit on train data to maintain consistency
    X_test[col] = X_test[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

# Ensure all columns are numeric
print(X_test.dtypes.value_counts())  # Should show only int64 and float64

# Make Predictions
final_preds = np.expm1(xgb.predict(X_test))  # Convert log back to original scale

# Create submission file
submission = pd.DataFrame({"Id": test["Id"], "SalePrice": final_preds})
submission.to_csv("submission.csv", index=False)

print("Submission file saved successfully!")

int32      36
int64      33
float64    12
Name: count, dtype: int64
Submission file saved successfully!
